In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import duckdb

os.chdir(r"C:\Users\siddu\Desktop\Decision Science Track\Revision")
print(os.getcwd())

In [ ]:
# Filepaths
train_data_fp = r"C:\Users\siddu\Desktop\Decision Science Track\round 2 files\train_data.parquet"
test_data_fp = r"C:\Users\siddu\Desktop\Decision Science Track\round 2 files\test_data.parquet"
event_data_fp = r"C:\Users\siddu\Desktop\Decision Science Track\round 2 files\add_event.parquet"
transaction_data_fp = r"C:\Users\siddu\Desktop\Decision Science Track\round 2 files\add_trans.parquet"
offer_metadata_fp = r"C:\Users\siddu\Desktop\Decision Science Track\round 2 files\offer_metadata.parquet"

In [ ]:
# Load files into dataframes
train_data_df = pd.read_parquet(train_data_fp)
test_data_df = pd.read_parquet(test_data_fp)
event_data_df = pd.read_parquet(event_data_fp)
transaction_data_df = pd.read_parquet(transaction_data_fp)
offer_metadata_df = pd.read_parquet(offer_metadata_fp)

In [ ]:
# Check for number of rows and columns
print(train_data_df.shape)
print(test_data_df.shape)
print(event_data_df.shape)
print(transaction_data_df.shape)
print(offer_metadata_df.shape)

In [ ]:
train_data_df.head()

In [ ]:
test_data_df.head()

In [ ]:
event_data_df.head()

In [ ]:
transaction_data_df.head()

In [ ]:
offer_metadata_df.head()

In [ ]:
# Normalize all id columns

def normalize_key_columns(df_dict, cols_to_normalize):
    """
    Iterates through a dictionary of dataframes and normalizes
    a specific list of columns (if they exist) by:
    1. Converting to string
    2. Stripping whitespace
    3. Converting to lowercase
    """
    print("--- Normalizing Key Columns ---")
    for df_name, df in df_dict.items():
        # Get a list of columns that are in BOTH the df and our list
        cols_in_df = [col for col in cols_to_normalize if col in df.columns]
        
        if not cols_in_df:
            # Skip if this df has none of the key columns
            continue

        print(f"Checking DataFrame: '{df_name}'. Normalizing: {cols_in_df}")
        
        for col in cols_in_df:
            # This .astype(str).str.strip().str.lower() chain
            # is a robust way to clean key columns.
            df[col] = df[col].astype(str).str.strip().str.lower()
            
    print("--- Normalization Complete ---")

# --- Run this right after loading your data ---

# 1. (You already have this) Load your DataFrames
# train_data_df = pd.read_parquet(train_data_fp)
# ...etc.

# 2. Put them in the dictionary (like we did before)
dataframes_to_check = {
    'train': train_data_df,
    'test': test_data_df,
    'event': event_data_df,
    'transaction': transaction_data_df,
    'offer_metadata': offer_metadata_df
}

# 3. Define all your known key columns
cols_to_normalize = ['id1', 'id2', 'id3', 'id4','id5','id6',
                       'id7','id8','id9','id10','id11','id12','id13']

# 4. Run the function
normalize_key_columns(dataframes_to_check, cols_to_normalize)

# Now, all your key columns are clean, and you can
# proceed with your analysis and intersections confidently.

In [ ]:
from collections import defaultdict


cols_to_check = ['id1', 'id2', 'id3', 'id4','id5','id6','id7','id8','id9','id10','id11','id12','id13']

# --- Assuming your setup from the prompt ---
# (You already have these lines)
# cols_to_check = ['id1', 'id2', 'id3', 'id4','id5','id6','id7','id8','id9','id10','id11','id12','id13']
# train_data_df = pd.read_parquet(train_data_fp)
# test_data_df = pd.read_parquet(test_data_fp)
# event_data_df = pd.read_parquet(event_data_fp)
# transaction_data_df = pd.read_parquet(transaction_data_fp)
# offer_metadata_df = pd.read_parquet(offer_metadata_fp)
# -----------------------------------------------


## 1. Group DataFrames for easy checking
# Use a dictionary to map a name to the DataFrame object
dataframes_to_check = {
    'train': train_data_df,
    'test': test_data_df,
    'event': event_data_df,
    'transaction': transaction_data_df,
    'offer_metadata': offer_metadata_df
}

## 2. Store findings in a dictionary
# The key will be the column name (e.g., 'id1')
# The value will be a set of all dtypes found for that column
column_dtypes = defaultdict(set)

## 3. Iterate and check
print("Checking column data types...")
for col in cols_to_check:
    for df_name, df in dataframes_to_check.items():
        # This is the key: check if the column exists in the DataFrame
        if col in df.columns:
            # If it exists, add its dtype (as a string) to the set
            column_dtypes[col].add(str(df[col].dtype))

## 4. Report the results
print("\n--- Dtype Consistency Report ---")
inconsistent_cols = []

for col, dtypes in column_dtypes.items():
    if len(dtypes) == 0:
        # This case means the column was in cols_to_check but not in any DataFrame
        print(f"ℹ️ {col:<15} | Not found in any dataset.")
    elif len(dtypes) == 1:
        # Only one dtype was found, so it's consistent
        print(f"✅ {col:<15} | Consistent: {list(dtypes)[0]}")
    else:
        # More than one dtype was found, this is an inconsistency
        print(f"⚠️ {col:<15} | INCONSISTENT: {dtypes}")
        inconsistent_cols.append(col)

print("\n--- Summary ---")
if not inconsistent_cols:
    print("All checked columns are consistent across datasets.")
else:
    print(f"Found {len(inconsistent_cols)} inconsistent columns: {inconsistent_cols}")

In [ ]:
# Get columns present in all datasets
train_data_df_cols = train_data_df.columns
test_data_df_cols = test_data_df.columns
event_data_df_cols = event_data_df.columns
transaction_data_df_cols = transaction_data_df.columns
offer_metadata_df_cols = offer_metadata_df.columns

# Find different in train and test data
print("Different columns in train and test data:")
for col in train_data_df_cols.difference(test_data_df_cols):
    print(col)

# Only 'y'-'clicked' was not present in the test data.

# Find common columns in train and event data
print("Common columns in train and event data:")
for col in train_data_df_cols.intersection(event_data_df_cols):
    print(col)

# Find common columns in train and transaction data
print("Common columns in train and transaction data:")
for col in train_data_df_cols.intersection(transaction_data_df_cols):
    print(col)

# Find common columns in train and offer data
print("Common columns in train and offer data:")
for col in train_data_df_cols.intersection(offer_metadata_df_cols):
    print(col)

In [ ]:
# Find common id2 between train and event data
common_id2 = set(train_data_df["id2"]).intersection(event_data_df["id2"])
print(f"Number of common id2s between train and event data: {len(common_id2)}")

# Find common id3 between train and event data
common_id3 = set(train_data_df["id3"]).intersection(event_data_df["id3"])
print(f"Number of common id3s between train and event data: {len(common_id3)}")

# Find common id4 between train and event data
common_id4 = set(train_data_df["id4"]).intersection(event_data_df["id4"])
print(f"Number of common id4s between train and event data: {len(common_id4)}")

# Find common id2 between test and event data
common_id2 = set(test_data_df["id2"]).intersection(event_data_df["id2"])
print(f"Number of common id2s between test and event data: {len(common_id2)}")

# Find common id3 between test and event data
common_id3 = set(test_data_df["id3"]).intersection(event_data_df["id3"])
print(f"Number of common id3s between test and event data: {len(common_id3)}")

# Find common id4 between test and event data
common_id4 = set(test_data_df["id4"]).intersection(event_data_df["id4"])
print(f"Number of common id4s between test and event data: {len(common_id4)}")



In [ ]:
# Find common id2 between train and test data
common_id2 = set(train_data_df["id2"]).intersection(test_data_df["id2"])
print(f"Number of common id2s between train and test data: {len(common_id2)}")

only_train_id2 = set(train_data_df["id2"]).difference(common_id2)
print(f"Number of unique id2 in train and not in test:  {len(only_train_id2)}")

only_test_id2 = set(test_data_df["id2"]).difference(common_id2)
print(f"Number of unique id2 in test and not in train:  {len(only_test_id2)}")

train_id2 = set(train_data_df["id2"])
print(f'Number of unique id2 in train: {len(train_id2)}')

test_id2 = set(test_data_df["id2"])
print(f'Number of unique id2 in test: {len(test_id2)}')

In [ ]:
# Find common id3 between train and test data
common_id3 = set(train_data_df["id3"]).intersection(test_data_df["id3"])
print(f"Number of common id3s between train and test data: {len(common_id3)}")

only_train_id3 = set(train_data_df["id3"]).difference(common_id3)
print(f"Number of unique id3 in train and not in test:  {len(only_train_id3)}")

only_test_id3 = set(test_data_df["id3"]).difference(common_id3)
print(f"Number of unique id3 in test and not in train:  {len(only_test_id3)}")

train_id3 = set(train_data_df["id3"])
print(f'Number of unique id3 in train: {len(train_id3)}')

test_id3 = set(test_data_df["id3"])
print(f'Number of unique id3 in test: {len(test_id3)}')

In [ ]:
# Find common id4 between train and test data
common_id4 = set(train_data_df["id4"]).intersection(test_data_df["id4"])
print(f"Number of common id4 between train and test data: {len(common_id4)}")

only_train_id4 = set(train_data_df["id4"]).difference(common_id4)
print(f"Number of unique id4 in train and not in test:  {len(only_train_id4)}")

only_test_id4 = set(test_data_df["id4"]).difference(common_id4)
print(f"Number of unique id4 in test and not in train:  {len(only_test_id4)}")

train_id4 = set(train_data_df["id4"])
print(f'Number of unique id4 in train: {len(train_id4)}')

test_id4 = set(test_data_df["id4"])
print(f'Number of unique id4 in test: {len(test_id4)}')

In [ ]:
# Find common id2 between train and transaction data
common_id2_train_txn = set(train_data_df["id2"]).intersection(transaction_data_df["id2"])
print(f"Number of common id2s between train and transaction data: {len(common_id2_train_txn)}")

In [ ]:
# Find common id2 between train and event data
common_id2_train_event = set(train_data_df["id2"]).intersection(event_data_df["id2"])
print(f"Number of common id2s between train and event data: {len(common_id2_train_event)}")

In [ ]:
# Find common id2 between transaction and event data
common_id2_transaction_event = set(transaction_data_df["id2"]).intersection(event_data_df["id2"])
print(f"Number of common id2s between transaction and event data: {len(common_id2_transaction_event)}")

In [ ]:
# Find common id3 between train and offer data
common_id3_train_offer = set(train_data_df["id3"]).intersection(offer_metadata_df["id3"])
print(f"Number of common id3s between train and offer data: {len(common_id3_train_offer)}")

In [ ]:
# Find common id2 between transaction and event data
common_id2 = set(transaction_data_df["id2"]).intersection(event_data_df["id2"])
print(f"Number of common id2 between transaction and event data: {len(common_id2)}")

only_transaction_id2 = set(transaction_data_df["id2"]).difference(common_id2)
print(f"Number of unique id2 in transaction and not in event:  {len(only_transaction_id2)}")

only_event_id2 = set(event_data_df["id2"]).difference(common_id2)
print(f"Number of unique id2 in event and not in transaction:  {len(only_event_id2)}")

transaction_id2 = set(transaction_data_df["id2"])
print(f'Number of unique id2 in transaction: {len(transaction_id2)}')

event_id2 = set(event_data_df["id2"])
print(f'Number of unique id2 in event: {len(event_id2)}')

Progress till now
- Merge offer with train and test
- Merge transaction with event (do later if needed)

In [ ]:
# Merge offer with train and test on id3
train_data_merged_df = pd.merge(train_data_df, offer_metadata_df, how="left", on='id3')
test_data_merged_df = pd.merge(test_data_df, offer_metadata_df, how="left", on='id3')

print(train_data_merged_df.head())
print(test_data_merged_df.head())


In [ ]:
# Check data integrity

# Check for number of rows and columns
print(train_data_merged_df.shape)
print(test_data_merged_df.shape)

# Get columns present in all datasets
train_data_merged_df_cols = train_data_merged_df.columns
test_data_merged_df_cols = test_data_merged_df.columns

# Find different in train_merged and test_merged data
print("Different columns in train_merged and test_merged data:")
for col in train_data_merged_df_cols.difference(test_data_merged_df_cols):
    print(col)

# Only 'y'-'clicked' was not present in the test_merged data.

In [ ]:
os.chdir(r"C:\Users\siddu\Desktop\Decision Science Track\Revision")
print(os.getcwd())

In [ ]:
import pandas as pd

# --- Assuming your merged DataFrames are in memory ---
# train_data_merged_df = pd.merge(train_data_df, offer_metadata_df, how="left", on='id3')
# test_data_merged_df = pd.merge(test_data_df, offer_metadata_df, how="left", on='id3')

# Define your output filepaths
train_output_fp = 'train_data_merged.parquet'
test_output_fp = 'test_data_merged.parquet'

print(f"Saving merged training data to {train_output_fp}...")
# Use .to_parquet() to save
# index=False is important to avoid saving the pandas index as a separate column
train_data_merged_df.to_parquet(train_output_fp, index=False)

print(f"Saving merged test data to {test_output_fp}...")
test_data_merged_df.to_parquet(test_output_fp, index=False)

print("Save complete.")

In [ ]:
train_data_merged_df['f285'].dtype

In [ ]:
import pandas as pd

# Assuming your DataFrame is loaded into a variable named 'train_data_merged_df'
# Example (uncomment and use if you need to load it):
# train_data_merged_df = pd.read_csv('your_file_name.csv')

# Find columns where all values are NaN
all_nan_cols = train_data_merged_df.columns[train_data_merged_df.isnull().all()]

# Get the count of such columns
num_all_nan_cols = len(all_nan_cols)

# Print the results
if num_all_nan_cols > 0:
    print(f"Found {num_all_nan_cols} columns with only NaN values.")
    print("These columns are:")
    
    # Print the list of column names
    for col_name in all_nan_cols:
        print(col_name)
else:
    print("No columns were found with only NaN values.")